# Calculating discounted free cash flow using neural networks

This model uses neural networks (NN) to estimate the future free cash flow of a company from the past financial statements.<br>
Specifically, we train the NN with the annual changes of the company's financial state over 4 years to project the future free cash flow (3 years).

**This code runs on the [Quantconnct](https://www.quantconnect.com) cloud.**<br>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce
import torch
from torch import nn
import pandas as pd


def grab_data(line_item0,line_item1):
    global start_time, end_time
    line_item = "FinancialStatements." +line_item0+"."+line_item1+".TwelveMonths"
    out = qb.GetFundamental(qb.Securities.Keys, line_item, start_time, end_time).drop_duplicates()[1:]
    out.columns = [line_item1]
    return out

def discounted_cash_flow(FCF,wacc,pgr):
    # discount future cash flow to the present value
    m = len(FCF)
    dFCF = np.zeros(m+1)
    for i in range(m):
        dFCF[i] = FCF[i]/(1+wacc)**i
    dFCF[-1] = FCF[-1]*(1+pgr)/(wacc-pgr)b
    return dFCF

qb = QuantBook()
ticker = "AAPL"
symbol = qb.AddEquity(ticker).Symbol
price  = qb.History(qb.Symbol(ticker), 1, Resolution.Daily)['close'].values[0]

end_time = datetime.now()
start_time = end_time - 11*timedelta(days=365) # dates back to roughtly 10 years

In [ ]:
# Loading the relevant line items of the financial statements
Data = grab_data("IncomeStatement","NetIncomeCommonStockholders")
Data = Data.add(grab_data("IncomeStatement","TotalRevenue"),fill_value=0)
Data = Data.add(grab_data("IncomeStatement","GrossProfit"),fill_value=0)
Data = Data.add(grab_data("CashFlowStatement","DepreciationAndAmortization"),fill_value=0)
Data = Data.add(grab_data("CashFlowStatement","ChangeInWorkingCapital"),fill_value=0)
Data = Data.add(grab_data("CashFlowStatement","DeferredTax"),fill_value=0)
Data = Data.add(grab_data("CashFlowStatement","OperatingCashFlow"),fill_value=0)
# Data = Data.add(grab_data("CashFlowStatement","StockBasedCompensation"),fill_value=0)
Data = Data.add(grab_data("CashFlowStatement","CapitalExpenditure"),fill_value=0)
Data = Data.add(grab_data("BalanceSheet","OrdinarySharesNumber"),fill_value=0)
Data = Data.add(grab_data("BalanceSheet","CurrentAssets"),fill_value=0)
Data = Data.add(grab_data("BalanceSheet","CurrentLiabilities"),fill_value=0)
Data = Data.add(grab_data("BalanceSheet","TotalDebt"),fill_value=0)
Data = Data.add(grab_data("BalanceSheet","TangibleBookValue"),fill_value=0)
# Data = Data.add(grab_data("BalanceSheet","TotalLiabilitiesAsReported"),fill_value=0)
Data = Data.add(grab_data("BalanceSheet","CashAndCashEquivalents"),fill_value=0)
Data = Data.add(grab_data("BalanceSheet","CommonStockEquity"),fill_value=0)
Data = Data.fillna(0)

# Calculate free cash flow
# Note that this calculatation excludes the stock bashed compensation, which otherwise inflates the free cash flow.
FCF = pd.DataFrame(Data[['NetIncomeCommonStockholders','DepreciationAndAmortization',
                         'ChangeInWorkingCapital','DeferredTax','CapitalExpenditure']].sum(axis=1),
                   columns=['FreeCashFlow'])

In [ ]:
# Select traning input data
Select   = Data[['NetIncomeCommonStockholders','TotalRevenue','GrossProfit','OperatingCashFlow'
                 ,'CashAndCashEquivalents','CommonStockEquity','CurrentLiabilities','CurrentAssets'
                ,'TotalDebt','TangibleBookValue']]
# Calculate annual changes
Del_data = (Select.values[1:] - Select.values[:-1])/Select.values[:-1]
# Cleaning up irregularities (np.nan and np.inf)
Del_data[Del_data==np.inf] = 0
Del_data = np.nan_to_num(Del_data)
# Calculate annual changes of free cash flow
Del_FCF  = (FCF.values[1:] - FCF.values[:-1])/np.abs(FCF.values[:-1])

In [ ]:
# Construct neural networks (NN) with pytorch
# We have 30 input nodes, one hidden layer with 15 nodes, and an output layer with 3 nodes (three years of projection).
# For the activation function, we use ReLu.
class Networks(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(30, 15),
      nn.ReLU(),
      nn.Linear(15, 15),
      nn.ReLU(),
      nn.Linear(15, 3)
    )


  def forward(self, x):
    return self.layers(x)

In [ ]:
model = Networks()
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Run the training loop
for epoch in range(0, 30): # 5 epochs at maximum
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    # Set current loss value
    current_loss = 0.0

    for i in range(Del_data.shape[0]-5):
        input_data = torch.from_numpy(Del_data[i:i+3]).flatten().float()
        target     = torch.from_numpy(Del_FCF[i+3:i+6]).flatten().float()
         
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = model.forward(input_data)

        # Compute loss
        loss = loss_function(outputs, target)
        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

print('Training process has finished.')

In [ ]:
# Predict the next three years of growth rates of future free cash flow with the last three annual changes (4 years worth of data)
new_data = Del_data[-3:].flatten()
new_data = torch.from_numpy(new_data).float()
predict = np.array(model.forward(new_data).detach())

# Compute the future free cash flow from the predicted growth rates.
temp = FCF.values[-1][0]
eFCF = []
for i in range(predict.shape[0]):
    temp *= (1+ predict[i])
    eFCF.append(temp)

In [ ]:
# Comupte the present value
Present_value = np.sum(discounted_cash_flow(eFCF,0.1,0.025))
sout = Data["OrdinarySharesNumber"].values[-1] # shares outstanding
# Compute a fair value of equity
Fair_value = Present_value/sout
# margin of safty (50%)
MOS = 0.5

In [ ]:
# Plotting the projected free cash flow
N = Data.index.year.values
NN = np.arange(N[-1]+1,N[-1]+1+len(predict),1)

anno_str = 'Current value of equity = {:.2f} USD'.format(price)+'\n'
anno_str += 'Fair value of equity = {:.2f} USD'.format(Fair_value)+'\n'
anno_str += 'Margin of safety = {:.2f}%'.format(MOS*100)+'\n'
anno_str += 'Ideal entry price = {:.2f} USD'.format(Fair_value*MOS)+'\n'
anno_str += 'Upside potential = {:.2f}%'.format((Fair_value*MOS/price-1)*100)+'\n'
anno_str += '\n'
anno_str += '\n'
anno_str += 'Perpetual growth rate = {:0.2f}%'.format(0.025*100)+'\n'
anno_str += 'Required rate of return = {:0.2f}%'.format(0.1*100)+'\n'

today = date.today()
d1 = today.strftime("%d/%m/%Y")

plt.figure(figsize=(8,6))
plt.bar(N,FCF.values.T[0]/1e9, label = 'Past')
plt.bar(NN,np.array(eFCF)/1e9, label = 'Future')
plt.legend(fontsize=14)
plt.xlabel('Year',fontsize=14)
plt.ylabel('Free Cash Flow [Billion USD]',fontsize=14)
plt.annotate(anno_str,
            xy = (1.01,0.95), xycoords='axes fraction', ha='left',va='top' ,fontsize =12)
plt.title(ticker+'  '+d1)
plt.show()